In [1]:
from rag_retrieval import retrieve_context,get_dates_documents,build_context
from database_manager import MySQLDB,ChromaDBConnector


/Users/rodolfocacacho/miniforge3/envs/bundesbaubot/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
import pandas as pd
import os

In [3]:
cwd_path = '/Users/rodolfocacacho/Documents/Documents/MAI/Master Thesis/Code/rag_clean'
os.chdir(cwd_path)

In [4]:
DIST_THRESHOLD = 250
DATE_DB_FILE = 'data/documents/metadata/Files_date_version.csv'

path_store ='/Users/rodolfocacacho/Documents/Documents/MAI/Master Thesis/Code/rag_clean/data/storage/embeddings'
con = ChromaDBConnector(storage_path=path_store,collection_name='rag_faq2')

# MySQL connection details
CONFIG_SQL_DB = {
    'user': 'root',
    'password': 'admin123',
    'host': 'localhost'
}

DB_NAME = 'data_rag'
TABLE_NAME = 'embedding_table'

sql_db_connector = MySQLDB(CONFIG_SQL_DB,DB_NAME)

MX_RESULTS = 10
date_project = '10/04/2024'
date_database = pd.read_csv(DATE_DB_FILE)


Some weights of BertModel were not initialized from the model checkpoint at aari1995/German_Semantic_V3 and are newly initialized: ['embeddings.position_embeddings.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Collection rag_faq2 already exists with 19038 documents.


In [5]:
available_documents = date_database['type'].unique()
# available_documents = ['Liste förderfähigen Anlagen - Wärmepumpen']

dates_files = get_dates_documents(date_project,date_database)

for i in dates_files:
    doc = i['type']
    dates = i['dates']
    type_key = i['type_key']
    temp_and = {"$and": [{"valid_date":{'$in':dates}},{"type_key":{'$eq':type_key}}]}
    print(f'{doc} - {dates} - {type_key}')

Technische FAQ BEG - ['13/03/2023'] - 1
Richtlinie BEG EM - ['21/12/2023'] - 2
Allgemeines Merkblatt zur Antragstellung - ['01/04/2024'] - 3
FAQ BEG (EM und EH-EG) - ['26/02/2024'] - 4
Infoblätter förderfähigen Kosten - ['01/01/2024'] - 6
Liste förderfähigen Anlagen - Wärmepumpen - ['01/04/2024'] - 5
Förderübersicht BEG EM - ['01/01/2024'] - 7
Liste förderfähigen Anlagen - Biomasse - ['01/01/2024'] - 8


In [6]:
query = 'Welche Maßnahmen zur Fachplanung und Baubegleitung sind förderfähig?'

results = retrieve_context(chroma_connector=con,
                            sql_connector=sql_db_connector,
                            sql_table=TABLE_NAME,
                            query=query,
                            max_results=MX_RESULTS,
                            print_t=False,
                            distance_threshold=800,
                            retrieve_date_documents = dates_files)

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


{'$or': [{'$and': [{'valid_date': {'$in': ['13/03/2023']}}, {'type_key': {'$eq': 1}}]}, {'$and': [{'valid_date': {'$in': ['21/12/2023']}}, {'type_key': {'$eq': 2}}]}, {'$and': [{'valid_date': {'$in': ['01/04/2024']}}, {'type_key': {'$eq': 3}}]}, {'$and': [{'valid_date': {'$in': ['26/02/2024']}}, {'type_key': {'$eq': 4}}]}, {'$and': [{'valid_date': {'$in': ['01/01/2024']}}, {'type_key': {'$eq': 6}}]}, {'$and': [{'valid_date': {'$in': ['01/04/2024']}}, {'type_key': {'$eq': 5}}]}, {'$and': [{'valid_date': {'$in': ['01/01/2024']}}, {'type_key': {'$eq': 7}}]}, {'$and': [{'valid_date': {'$in': ['01/01/2024']}}, {'type_key': {'$eq': 8}}]}]}
0 retrieved, []


In [16]:
def compare_texts(text1, text2):
    differences = []
    
    # Find the length of the shorter text to avoid index errors
    min_len = min(len(text1), len(text2))
    
    # Compare each character in the two texts
    for i in range(min_len):
        if text1[i] != text2[i]:
            differences.append((i, text1[i], text2[i]))
    
    # Check if one text is longer than the other
    if len(text1) > len(text2):
        for i in range(min_len, len(text1)):
            differences.append((i, text1[i], None))
    elif len(text2) > len(text1):
        for i in range(min_len, len(text2)):
            differences.append((i, None, text2[i]))

    return differences

In [20]:
text_db = 'Infoblätter förderfähigen Kosten'
text_file = 'Infoblätter förderfähigen Kosten'

differences = compare_texts(text_db,text_file)

for diff in differences:
    print(f"Difference at index {diff[0]}: '{diff[1]}' vs '{diff[2]}'")

Difference at index 6: 'a' vs 'ä'
Difference at index 7: '̈' vs 't'
Difference at index 9: 't' vs 'e'
Difference at index 10: 'e' vs 'r'
Difference at index 11: 'r' vs ' '
Difference at index 12: ' ' vs 'f'
Difference at index 13: 'f' vs 'ö'
Difference at index 14: 'o' vs 'r'
Difference at index 15: '̈' vs 'd'
Difference at index 16: 'r' vs 'e'
Difference at index 17: 'd' vs 'r'
Difference at index 18: 'e' vs 'f'
Difference at index 19: 'r' vs 'ä'
Difference at index 20: 'f' vs 'h'
Difference at index 21: 'a' vs 'i'
Difference at index 22: '̈' vs 'g'
Difference at index 23: 'h' vs 'e'
Difference at index 24: 'i' vs 'n'
Difference at index 25: 'g' vs ' '
Difference at index 26: 'e' vs 'K'
Difference at index 27: 'n' vs 'o'
Difference at index 28: ' ' vs 's'
Difference at index 29: 'K' vs 't'
Difference at index 30: 'o' vs 'e'
Difference at index 31: 's' vs 'n'
Difference at index 32: 't' vs 'None'
Difference at index 33: 'e' vs 'None'
Difference at index 34: 'n' vs 'None'


In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("amberoad/bert-multilingual-passage-reranking-msmarco")
model = AutoModelForSequenceClassification.from_pretrained("amberoad/bert-multilingual-passage-reranking-msmarco")

# Define the query and candidate passages
query = "What is the capital of Germany?"
passages = [
    "Berlin is the capital of Germany.",
    "Paris is the capital of France.",
    "Madrid is the capital of Spain."
]

# Tokenize and encode the query and passages
inputs = [f"[CLS] {query} [SEP] {passage} [SEP]" for passage in passages]
encoded_inputs = tokenizer(inputs, padding=True, truncation=True, return_tensors="pt")

# Pass the encoded inputs through the model
with torch.no_grad():
    outputs = model(**encoded_inputs)

# Extract the scores (logits)
scores = outputs.logits

# Rank the passages based on the scores
ranked_passages = sorted(zip(passages, scores[:, 1].tolist()), key=lambda x: x[1], reverse=True)

# Print the ranked passages
for passage, score in ranked_passages:
    print(f"Score: {score:.4f} | Passage: {passage}")

/Users/rodolfocacacho/miniforge3/envs/bundesbaubot/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Score: -3.8218 | Passage: Berlin is the capital of Germany.
Score: -3.9309 | Passage: Madrid is the capital of Spain.
Score: -3.9837 | Passage: Paris is the capital of France.


In [35]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("deepset/gbert-base-germandpr-reranking")
model = AutoModelForSequenceClassification.from_pretrained("deepset/gbert-base-germandpr-reranking")

# Define the query and candidate passages (in German)
query = "Wie hoch ist der Mount Everest?"
passages = [
    "Der Mount Everest ist 8.848 Meter hoch.",  # Highly relevant
    "Mein Lieblingsfilm ist ein Actionfilm aus den 90ern.",  # Completely unrelated
]

# Tokenize and encode the query and passages
inputs = [f"[CLS] {query} [SEP] {passage} [SEP]" for passage in passages]
encoded_inputs = tokenizer(inputs, padding=True, truncation=True, return_tensors="pt")

# Pass the encoded inputs through the model
with torch.no_grad():
    outputs = model(**encoded_inputs)

# Extract the scores (logits)
scores = outputs.logits

# Rank the passages based on the scores
ranked_passages = sorted(zip(passages, scores[:, 1].tolist()), key=lambda x: x[1], reverse=True)

# Print the ranked passages
for passage, score in ranked_passages:
    print(f"Score: {score:.4f} | Passage: {passage}")

Score: -1.8674 | Passage: Der Mount Everest ist 8.848 Meter hoch.
Score: -2.3597 | Passage: Mein Lieblingsfilm ist ein Actionfilm aus den 90ern.


In [41]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("cross-encoder/ms-marco-MiniLM-L-6-v2")
model = AutoModelForSequenceClassification.from_pretrained("cross-encoder/ms-marco-MiniLM-L-6-v2")

# Define a query and passages
query = "Was ist die Hauptstadt von Frankreich?"
passages = [
    "Paris ist die Hauptstadt von Frankreich.",
    "Die Hauptstadt von Deutschland ist Berlin.",
    "Frankreich liegt in Europa und hat viele schöne Städte.",
    "Rom ist die Hauptstadt von Italien."
]

# Tokenize the query-passage pairs
inputs = [f"{query} [SEP] {passage}" for passage in passages]
encoded_inputs = tokenizer(inputs, padding=True, truncation=True, return_tensors="pt")

# Pass the inputs through the model
with torch.no_grad():
    outputs = model(**encoded_inputs)

# Extract logits (single score for each query-passage pair)
scores = outputs.logits.squeeze()

# Rank passages based on the scores
ranked_passages = sorted(zip(passages, scores.tolist()), key=lambda x: x[1], reverse=True)

# Print the ranked passages
for passage, score in ranked_passages:
    print(f"Score: {score:.4f} | Passage: {passage}")

Score: -5.2375 | Passage: Die Hauptstadt von Deutschland ist Berlin.
Score: -5.3239 | Passage: Paris ist die Hauptstadt von Frankreich.
Score: -7.8500 | Passage: Rom ist die Hauptstadt von Italien.
Score: -8.7271 | Passage: Frankreich liegt in Europa und hat viele schöne Städte.


In [45]:
from sentence_transformers import SentenceTransformer, util

# Load the model
model = SentenceTransformer("Alibaba-NLP/gte-multilingual-reranker-base",trust_remote_code=True)

# Define a query and passages
query = "Wie heißt der höchste Berg der Welt?"
passages = [
    "Der Mount Everest ist der höchste Berg der Welt.",
    "Der K2 ist der zweithöchste Berg der Welt.",
    "Das Himalaya-Gebirge liegt in Asien.",
    "Der Eiffelturm ist ein berühmtes Wahrzeichen in Paris."
]
# Encode the query and passages
query_embedding = model.encode(query, convert_to_tensor=True)
passage_embeddings = model.encode(passages, convert_to_tensor=True)

# Compute cosine similarity between the query and each passage
cosine_scores = util.cos_sim(query_embedding, passage_embeddings)

# Rank passages based on similarity scores
ranked_passages = sorted(zip(passages, cosine_scores[0].tolist()), key=lambda x: x[1], reverse=True)

# Print the ranked passages with their scores
for passage, score in ranked_passages:
    print(f"Score: {score:.4f} | Passage: {passage}")

No sentence-transformers model found with name Alibaba-NLP/gte-multilingual-reranker-base. Creating a new one with mean pooling.


Score: 0.9363 | Passage: Das Himalaya-Gebirge liegt in Asien.
Score: 0.8424 | Passage: Der K2 ist der zweithöchste Berg der Welt.
Score: 0.6888 | Passage: Der Eiffelturm ist ein berühmtes Wahrzeichen in Paris.
Score: 0.6751 | Passage: Der Mount Everest ist der höchste Berg der Welt.


# Test Reranking

In [1]:
from rag_retrieval import retrieve_context,get_dates_documents,build_context
from database_manager import MySQLDB,ChromaDBConnector
import os
import pandas as pd

cwd_path = '/Users/rodolfocacacho/Documents/Documents/MAI/Master Thesis/Code/rag_clean'
os.chdir(cwd_path)

DIST_THRESHOLD = 600
DATE_DB_FILE = 'data/documents/metadata/Files_date_version.csv'

path_store ='/Users/rodolfocacacho/Documents/Documents/MAI/Master Thesis/Code/rag_clean/data/storage/embeddings'
con = ChromaDBConnector(storage_path=path_store,collection_name='rag_faq2')

# MySQL connection details
CONFIG_SQL_DB = {
    'user': 'root',
    'password': 'admin123',
    'host': 'localhost'
}

DB_NAME = 'data_rag'
TABLE_NAME = 'embedding_table'

sql_db_connector = MySQLDB(CONFIG_SQL_DB,DB_NAME)

MX_RESULTS = 6
MX_RESULTS_QUERY = 25
date_project = '10/04/2024'
date_database = pd.read_csv(DATE_DB_FILE)

available_documents = date_database['type'].unique()
# available_documents = ['Liste förderfähigen Anlagen - Wärmepumpen']

dates_files = get_dates_documents(date_project,date_database)

for i in dates_files:
    doc = i['type']
    dates = i['dates']
    type_key = i['type_key']
    temp_and = {"$and": [{"valid_date":{'$in':dates}},{"type_key":{'$eq':type_key}}]}
    print(f'{doc} - {dates} - {type_key}')

def rerank_documents_by_distance_and_model(documents, query):
    # Load the model
    model = SentenceTransformer("Alibaba-NLP/gte-multilingual-reranker-base",trust_remote_code=True)

    # Step 1: Order documents by distance (smaller distance gets higher rank)
    documents_sorted_by_distance = sorted(documents, key=lambda x: x['distance'])
    
    # Assign original ranking based on distance
    for rank, doc in enumerate(documents_sorted_by_distance, start=1):
        doc['org_ranking'] = rank

    # Step 2: Extract content for re-ranking
    passages = [doc['content'] for doc in documents_sorted_by_distance]

    # Step 3: Encode the query and passages
    query_embedding = model.encode(query, convert_to_tensor=True)
    passage_embeddings = model.encode(passages, convert_to_tensor=True)

    # Step 4: Compute cosine similarity between the query and each passage
    cosine_scores = util.cos_sim(query_embedding, passage_embeddings)

    # Step 5: Rank documents based on similarity scores
    ranked_documents = sorted(zip(documents_sorted_by_distance, cosine_scores[0].tolist()), key=lambda x: x[1], reverse=True)

    # Step 6: Assign new ranking based on the reranker
    for new_rank, (doc, score) in enumerate(ranked_documents, start=1):
        doc['new_ranking'] = new_rank

    return documents_sorted_by_distance



/Users/rodolfocacacho/miniforge3/envs/bundesbaubot/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
Some weights of BertModel were not initialized from the model checkpoint at aari1995/German_Semantic_V3 and are newly initialized: ['embeddings.position_embeddings.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Collection rag_faq2 already exists with 19038 documents.
Technische FAQ BEG - ['13/03/2023'] - 1
Richtlinie BEG EM - ['21/12/2023'] - 2
Allgemeines Merkblatt zur Antragstellung - ['01/04/2024'] - 3
FAQ BEG (EM und EH-EG) - ['26/02/2024'] - 4
Infoblätter förderfähigen Kosten - ['01/01/2024'] - 6
Liste förderfähigen Anlagen - Wärmepumpen - ['01/04/2024'] - 5
Förderübersicht BEG EM - ['01/01/2024'] - 7
Liste förderfähigen Anlagen - Biomasse - ['01/01/2024'] - 8


In [3]:
query = 'Welche Maßnahmen zur Fachplanung und Baubegleitung sind förderfähig?'

results,ids1,ids2 = retrieve_context(chroma_connector=con,
                            sql_connector=sql_db_connector,
                            sql_table=TABLE_NAME,
                            query=query,
                            max_results=MX_RESULTS,
                            distance_threshold=800,
                            retrieve_date_documents = dates_files,
                            max_results_query=MX_RESULTS_QUERY,
                            distance_threshold_query=DIST_THRESHOLD)


No sentence-transformers model found with name Alibaba-NLP/gte-multilingual-reranker-base. Creating a new one with mean pooling.


Filtered out 19 documents after re-ranking.


In [4]:
results

[{'id': '19.187',
  'content': 'Entsprechende Nachhaltigkeits-Beraterinnen oder –Berater können bei Zertifizierungsstellen und Anbietenden von Bewertungssystemen des Nachhaltigen Bauens, die im Bereich des „Qualitätssiegels Nachhaltiges Gebäude“ tätig werden dürfen, nachgefragt werden.\nGrundsätzlich können auch entsprechend qualifizierte Planerinnen und Planer sowie Ingenieurinnen und Ingenieure sowie Mitarbeitende von Bauträgern, Fertighausunternehmen, Planungs- und Ingenieurbüros sowie Bauverwaltungen von Bund, Ländern und Kommunen die förderfähigen Beratungs- und Planungsleistungen der Nachhaltigkeitszertifizierung ausführen. Näheres hierzu regeln die Zertifizierungsstellen in eigener Zuständigkeit. Grundsätzlich ist es möglich, dass die Fördernehmenden mehr als einen Dienstleistenden mit Beratungs- und Planungsleistungen der Nachhaltigkeitszertifizierung beauftragen.\n6.1 Energetische Fachplanung und Baubegleitung\nDie nachfolgende Auflistung der förderfähigen Leistungen enthält s

In [5]:
from sentence_transformers import SentenceTransformer, util
import torch
import json

# Initialize your model
matryoshka_dim = 1024
model = SentenceTransformer("aari1995/German_Semantic_V3", trust_remote_code=True, truncate_dim=matryoshka_dim)

# Sample sentence pairs from your database for manual checking
sentence_pairs = [
    ("", ""),  # Expected to be similar
    ("Zwei Personen beobachten das Wasser.", "Der Hund läuft im Park."),  # Expected to be dissimilar
    ("Das Auto fährt schnell.", "Das Fahrzeug bewegt sich zügig."),  # Expected to be similar
    ("Das Wetter ist sonnig.", "Das Wasser ist kalt.")  # Expected to be dissimilar
]

# Define a function to evaluate the embeddings and similarities
def evaluate_embeddings(model, sentence_pairs):
    # Extract individual sentences from pairs
    sentences = [s for pair in sentence_pairs for s in pair]

    # Encode the sentences to embeddings
    embeddings = model.encode(sentences, convert_to_tensor=True).half()

    # Initialize a list to store the results
    results = []

    # Iterate over the sentence pairs
    for i, (sentence1, sentence2) in enumerate(sentence_pairs):
        # Get the indices of the sentences
        idx1 = i * 2
        idx2 = i * 2 + 1

        # Compute the cosine similarity
        similarity = util.cos_sim(embeddings[idx1], embeddings[idx2]).item()

        # Store the result
        results.append({
            "sentence1": sentence1,
            "sentence2": sentence2,
            "similarity": similarity
        })

    return results



# # Call the evaluation function and get the results
# results = evaluate_embeddings(model, sentence_pairs)

# # Print the results for manual inspection
# for result in results:
#     print(f"Sentence 1: {result['sentence1']}")
#     print(f"Sentence 2: {result['sentence2']}")
#     print(f"Similarity Score: {result['similarity']:.4f}\n")

In [5]:
path_store ='/Users/rodolfocacacho/Documents/Documents/MAI/Master Thesis/Code/rag_clean/data/storage/embeddings'

con = ChromaDBConnector(storage_path=path_store,collection_name='rag_faq2')

model = SentenceTransformer("aari1995/German_Semantic_V3", trust_remote_code=True)

def evaluate_similarity(query, data, model):
    # Encode the query
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Extract all the contents (texts) from the data
    contexts = [item['content'] for item in data]

    # Encode all the contexts
    context_embeddings = model.encode(contexts, convert_to_tensor=True)

    # Compute the similarities between the query and all contexts
    similarities = util.cos_sim(query_embedding, context_embeddings)

    # Create a list to hold the results
    results = []

    # Loop over each context and its corresponding similarity score
    for idx, (context, similarity) in enumerate(zip(contexts, similarities[0])):
        results.append({
            "id": data[idx]["id"],
            "content": context,
            "similarity_score": similarity.item()
        })

    # Sort the results by similarity score in descending order
    results = sorted(results, key=lambda x: x['similarity_score'], reverse=True)

    return results

# MySQL connection details
CONFIG_SQL_DB = {
    'user': 'root',
    'password': 'admin123',
    'host': 'localhost'
}

DB_NAME = 'data_rag'
TABLE_NAME = 'embedding_table'

sql_db_connector = MySQLDB(CONFIG_SQL_DB,DB_NAME)

question = 'Welche technischen Anforderungen bestehen an Heizflächen?'

ids = ["1.183", "19.132", "63.1", "75.109", "19.166", "34.43", "34.207", "55.18", "34.206", "75.166", "19.176", "75.126", "19.136", "1.171", "19.169", "1.142", "75.129", "1.154", "75.169", "75.163", "1.130", "19.146", "1.151", "1.143", "19.164"]

Collection rag_faq2 already exists with 19038 documents.


NameError: name 'SentenceTransformer' is not defined

In [6]:
ids_2 = [
    '0.105',
'0.107',
'0.109',
'0.120',
'1.109',
'1.110',
'1.111',
'1.119',
'12.102',
'12.113',
'12.93',
'13.100',
'13.108',
'14.100',
'14.109',
'15.101',
'15.87',
'15.93',
'16.100',
'16.108',
'16.93',
'17.104',
'17.95',
'18.94',
'19.110',
'19.119',
'19.127',
'20.79',
'20.87',
'21.72',
'22.61',
'23.102',
'23.105',
'23.114',
'24.102',
'24.105',
'25.102',
'25.103',
'25.99',
'27.75',
'28.101',
'28.104',
'28.105',
'29.100',
'29.101',
'29.96',
'3.113',
'3.124',
'3.125',
'30.50',
'30.82',
'31.49',
'32.87',
'32.90',
'33.104',
'33.106',
'33.115',
'35.101',
'35.111',
'35.98',
'36.102',
'36.103',
'36.99',
'37.62',
'38.87',
'39.103',
'39.106',
'4.112',
'4.125',
'40.96',
'40.99',
'41.103',
'41.106',
'42.101',
'42.102',
'42.98',
'43.100',
'43.103',
'43.104',
'44.102',
'44.105',
'45.102',
'45.99',
'46.102',
'46.99',
'47.69',
'48.37',
'48.86',
'48.89',
'48.98',
'49.100',
'49.103',
'49.113',
'5.106',
'5.107',
'5.92',
'50.102',
'50.103',
'50.99',
'51.100',
'51.103',
'51.104',
'52.102',
'52.112',
'52.99',
'53.100',
'53.103',
'53.113',
'54.64',
'74.43',
'74.45',
'74.46',
'74.47',
'74.48',
'74.49',
'74.50',
'74.52',
'74.53',
'74.54',
'74.55',
'74.62',
'74.69',
'74.71',
'74.76',
'74.77',
'74.84',
'75.54',
'75.56',
'75.57',
'75.58',
'75.59',
'75.60',
'75.61',
'75.63',
'75.64',
'75.65',
'75.66',
'75.73',
'75.80',
'75.82',
'75.88',
'75.98',
'76.55',
'76.57',
'76.58',
'76.59',
'76.60',
'76.61',
'76.62',
'76.64',
'76.65',
'76.66',
'76.67',
'76.74',
'76.81',
'76.83',
'76.88',
'76.89',
'76.96',
'77.101',
'77.57',
'77.59',
'77.60',
'77.61',
'77.62',
'77.63',
'77.64',
'77.66',
'77.67',
'77.68',
'77.69',
'77.76',
'77.83',
'77.85',
'77.90',
'77.91',
'78.39',
'78.41',
'78.42',
'78.43',
'78.44',
'78.45',
'78.46',
'78.48',
'78.49',
'78.50',
'78.51',
'78.58',
'78.65',
'78.67',
'78.71',
'78.72',
'78.79'
]

In [23]:
def compare_new_embeddings_to_chroma(data_list, chroma_connector, model):
    """
    Compare embeddings of new documents (from data_list) to the stored embeddings in ChromaDB,
    using a specific model to generate embeddings for new documents.
    
    Args:
        data_list (list): A list of dictionaries containing 'id' and 'content'.
                          Example: [{'id': 'doc1', 'content': 'This is a document.'}, ...]
        chroma_connector (ChromaDBConnector): Instance of the ChromaDB connector.
        model (SentenceTransformer): The embedding model to generate embeddings for new documents.
    
    Returns:
        List of dictionaries: List of dictionaries containing 'id', 'content', and 'similarity'.
    """
    # Check if GPU is available and move tensors to the appropriate device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Extract IDs and contents from the list of dictionaries
    ids = [doc['id'] for doc in data_list]
    contents = [doc['content'] for doc in data_list]

    # Embed the new contents using the provided model and move them to the correct device
    new_embeddings = model.encode(contents, convert_to_tensor=True).to(device)

    # Retrieve the stored embeddings and content from ChromaDB using the document IDs
    stored_results = chroma_connector.col.get(ids=ids, include=['embeddings', 'documents'])

    # Extract stored embeddings and content
    stored_embeddings = stored_results['embeddings']
    stored_documents = stored_results['documents']

    # Convert the stored embeddings to a tensor and move them to the correct device
    stored_embeddings_tensor = torch.tensor(stored_embeddings).to(device)

    # Calculate cosine similarity between new embeddings and stored embeddings
    similarities = util.cos_sim(new_embeddings, stored_embeddings_tensor)

    # Print the results
    for idx in range(len(ids)):
        # Extract the diagonal elements (i.e., similarity between corresponding new and stored embeddings)
        similarity_score = similarities[idx][idx].item()
        print(f"Document ID: {ids[idx]} | Similarity: {similarity_score}")

    # Build a list of dictionaries to store the results
    results = []
    for idx in range(len(ids)):
        # Extract the diagonal elements (i.e., similarity between corresponding new and stored embeddings)
        similarity_score = similarities[idx][idx].item()

        # Append result to the list
        results.append({
            'id': ids[idx],
            'content': stored_documents[idx],
            'similarity': similarity_score
        })

    return results

In [24]:
ids_data = sql_db_connector.get_records_by_ids(table_name=TABLE_NAME,
                                    record_ids=ids)

for i in ids_data:
    metadata = json.loads(i['metadata'])
    pdf_name = metadata['source']
    title = pdf_name.replace('.pdf', '')
    chunk = i['content']
    new_chunk = f'''{title}\n\n{chunk}'''
    i['content'] = new_chunk

results = compare_new_embeddings_to_chroma(ids_data, con, model)

Document ID: 1.130 | Similarity: 0.8920531868934631
Document ID: 1.142 | Similarity: 0.803210437297821
Document ID: 1.143 | Similarity: 0.8320057392120361
Document ID: 1.151 | Similarity: 0.8784082531929016
Document ID: 1.154 | Similarity: 0.8328081965446472
Document ID: 1.171 | Similarity: 0.8882493376731873
Document ID: 1.183 | Similarity: 0.9104197025299072
Document ID: 19.132 | Similarity: 0.8450669646263123
Document ID: 19.136 | Similarity: 0.8510258793830872
Document ID: 19.146 | Similarity: 0.8754404783248901
Document ID: 19.164 | Similarity: 0.8498990535736084
Document ID: 19.166 | Similarity: 0.8859665393829346
Document ID: 19.169 | Similarity: 0.8487415313720703
Document ID: 19.176 | Similarity: 0.8488759994506836
Document ID: 34.206 | Similarity: 0.8190237283706665
Document ID: 34.207 | Similarity: 0.878131628036499
Document ID: 34.43 | Similarity: 0.8644486665725708
Document ID: 55.18 | Similarity: 0.8830145597457886
Document ID: 63.1 | Similarity: 0.8713582754135132
Docume

In [35]:
import unicodedata

def embedding_similarity(str1, str2, model_name="aari1995/German_Semantic_V3"):
    # Load a multilingual model that works well with German text
    model = SentenceTransformer(model_name)

    # Encode both strings to embeddings
    embeddings = model.encode([str1, str2], convert_to_tensor=True)

    # Compute cosine similarity between the two embeddings
    similarity = util.cos_sim(embeddings[0], embeddings[1])

    return similarity.item()

def normalize_text(text):
    # Normalize text to NFC (Canonical Composition)
    return unicodedata.normalize('NFC', text)

def check_encoding(text, encoding='utf-8'):
    # Check if the text can be encoded in the given encoding (e.g., UTF-8)
    try:
        text.encode(encoding)
        return True
    except UnicodeEncodeError:
        return False

def compare_texts(str1, str2):
    # Normalize and check encoding
    str1_normalized = normalize_text(str1)
    str2_normalized = normalize_text(str2)

    # Ensure both are UTF-8 encoded
    if not check_encoding(str1_normalized) or not check_encoding(str2_normalized):
        print("Encoding issue detected.")
        return None

    # Check for embedding-based similarity
    similarity_score = embedding_similarity(str1_normalized, str2_normalized)
    print(f"Similarity Score: {similarity_score:.4f}")
    return similarity_score

# Example Usage
str1 = "Schöne Grüße"
str2 = "Schoene Gruesse"
compare_texts(str1, str2)

Some weights of BertModel were not initialized from the model checkpoint at aari1995/German_Semantic_V3 and are newly initialized: ['embeddings.position_embeddings.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Similarity Score: 0.3555


0.355535089969635

In [34]:
for n,i in enumerate(results):
    db_data = i['content']
    sql_data = ids_data[n]['content']
    val = compare_texts(db_data, sql_data)
    
    # print(f'{i['content']}\n')
    # print(f'{ids_data[n]['content']}\n')

/Users/rodolfocacacho/miniforge3/envs/bundesbaubot/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Similarity Score: 1.0000
Similarity Score: 1.0000
Similarity Score: 1.0000
Similarity Score: 1.0000
Similarity Score: 1.0000
Similarity Score: 1.0000
Similarity Score: 1.0000
Similarity Score: 1.0000
Similarity Score: 1.0000
Similarity Score: 1.0000
Similarity Score: 1.0000
Similarity Score: 1.0000
Similarity Score: 1.0000
Similarity Score: 1.0000
Similarity Score: 1.0000
Similarity Score: 1.0000
Similarity Score: 1.0000
Similarity Score: 1.0000
Similarity Score: 1.0000
Similarity Score: 1.0000
Similarity Score: 1.0000
Similarity Score: 1.0000
Similarity Score: 1.0000
Similarity Score: 1.0000
Similarity Score: 1.0000


In [8]:
ids_data = sql_db_connector.get_records_by_ids(table_name=TABLE_NAME,
                                    record_ids=ids_2)

In [9]:
model = SentenceTransformer("aari1995/German_Semantic_V3", trust_remote_code=True)

query = 'Welchen U-Wert müssen Dachflächenfenster haben, um gefördert zu werden'
evaluate_similarity(query,ids_data,model)

[{'id': '23.102',
  'content': '4.18 Ist der Einbau von Fenstern mit Sonnenschutz im Scheibenzwischenraum im Rahmen des normalen Fensteraustauschs vollumfänglich förderfähig?\nDie Förderfähigkeit nach den technischen Mindestanforderungen der BEG EM Nummer 1.2 „Sommerlicher Wärmeschutz“ besteht für außenliegende Sonnenschutzeinrichtungen mit optimierter Tageslichtversorgung, z. B. über Lichtlenksysteme oder strahlungsabhängige Steuerung.\nDaher ist die Installation von nicht außenliegendem Sonnenschutz (d. h. innenliegend oder im Scheibenzwischenraum) sowie auch die Installation von außenliegendem Sonnenschutz ohne optimierte Tageslichtversorgung im Rahmen der Einzelmaßnahme „Sommerlicher Wärmeschutz“ nicht förderfähig.\nAllerdings sind Sonnenschutzmaßnahmen im Scheibenzwischenraum im Kontext der „Einzelmaßnahme Fenster" förderfähig, wenn die Fenster die jeweiligen Anforderungen an den U-Wert erfüllen.',
  'similarity_score': 0.7457820773124695},
 {'id': '24.102',
  'content': '4.18 Ist

In [10]:
ids_data

[{'id': '0.105',
  'content': 'Folgende Anforderungen an die Wärmedurchgangskoeffizienten (U-Werte) sind bei Sanierung der jeweiligen Bauteile für eine Förderung als Einzelmaßnahme einzuhalten. Die Anforderungen beziehen sich nur auf die wärmeübertragen-den Umfassungsflächen.',
  'metadata': '{"doc_type": "Richtlinie BEG EM", "page": 16, "type": "Text", "source": "Richtlinie BEG EM (2020-12-17).pdf", "path": "", "version": "X.X", "valid_date": "17/12/2020", "id": "0.105", "pre_id": "0.104", "post_id": "0.106"}'},
 {'id': '0.107',
  'content': 'Die Tabelle zeigt verschiedene Bauteilgruppen und ihre jeweiligen Wärmedurchgangskoeffizienten (U-Werte) in W/(m²·K) für unterschiedliche Bauarten und Materialien. \n\n**Überblick:**\n- Die Tabelle ist in zwei Hauptkategorien unterteilt: Außenwand und Bauteilgruppe Fenster, Fenstertüren, Dachflächenfenster, Glasdächer, Außentüren und Vorhangfassaden sowie Tore bei Nichtwohngebäuden.',
  'metadata': '{"doc_type": "Richtlinie BEG EM", "page": 16, "

In [13]:
def create_ids(y, x):
    """
    Creates a list of IDs in the format 'y.0', 'y.1', ..., 'y.x'.
    
    Args:
        y (str or int): The prefix for the IDs (e.g., 'y').
        x (int): The number of IDs to generate, from 0 to x.
    
    Returns:
        list: A list of strings in the format 'y.0', 'y.1', ..., 'y.x'.
    """
    return [f"{y}.{i}" for i in range(x + 1)]

ids_q1 = create_ids(75,172)

question = 'Welche technischen Anforderungen bestehen an wassergeführten Heizsystemen?'

ids_data = sql_db_connector.get_records_by_ids(table_name=TABLE_NAME,
                                    record_ids=ids_q1)

sims = evaluate_similarity(question,ids_data,model)
for n,i in enumerate(sims):
    print(i)
    if n > 4:
        print('')
        break

question = 'Welche Prüfanforderungen bestehen für Wärmepumpen?'

ids_data = sql_db_connector.get_records_by_ids(table_name=TABLE_NAME,
                                    record_ids=ids_q1)

sims = evaluate_similarity(question,ids_data,model)
for n,i in enumerate(sims):
    print(i)
    if n > 4:
        print('')
        break

question = 'Welche Anforderungen bestehen an den hydraulischen Abgleich bei Heizsystemen?'

ids_data = sql_db_connector.get_records_by_ids(table_name=TABLE_NAME,
                                    record_ids=ids_q1)

sims = evaluate_similarity(question,ids_data,model)
for n,i in enumerate(sims):
    print(i)
    if n > 4:
        print('')
        break


question = 'Welche Anforderungen gelten für neue Wärmeerzeuger in einem Gebäudenetz?'

ids_data = sql_db_connector.get_records_by_ids(table_name=TABLE_NAME,
                                    record_ids=ids_q1)

sims = evaluate_similarity(question,ids_data,model)
for n,i in enumerate(sims):
    print(i)
    if n > 4:
        print('')
        break


question = 'Was ist bei der Ertüchtigung von Fenstern zu beachten?'

ids_data = sql_db_connector.get_records_by_ids(table_name=TABLE_NAME,
                                    record_ids=ids_q1)

sims = evaluate_similarity(question,ids_data,model)
for n,i in enumerate(sims):
    print(i)
    if n > 4:
        print('')
        break


{'id': '75.114', 'content': 'Nr.: 8.03\nHauptthema: Anlagen zur Wärmeerzeugung (Heizungstechnik)\nThema/Stichwort: Hydraulischer Abgleich, wassergeführte Systeme', 'similarity_score': 0.7041393518447876}
{'id': '75.115', 'content': 'Beschreibung: Die Förderung von wassergeführten Heizungssystemen, die zur Raumheizung oder zur Raumheizungsunterstützung ausgelegt sind, setzt ein nach Verfahren B hydraulisch abgeglichenes Heizungssystem voraus. Der hydraulische Abgleich muss nach Verfahren B durchgeführt werden. Dieser ist bei Wohn- und Nichtwohngebäuden durch einen Fachunternehmer nach Verfahren B gemäß Bestätigungsformular des hydraulischen Abgleichs sowie der zugehörigen Fachregel des „VdZ-Wirtschaftsvereinigung Gebäude und Energie e. V.“ durchzuführen. Wurde in der Vergangenheit bereits ein hydraulischer Abgleich nach Verfahren B durchgeführt, ist die Verwendung vorhandener Berechnungen und Auslegungen möglich. Diese sind jedoch zu überprüfen und die Einstellungen an der Anlage ggf. n

In [15]:
ids_q1 = create_ids(34,291)

question = 'Welchen U-Wert müssen Dachflächenfenster haben, um gefördert zu werden'

ids_data = sql_db_connector.get_records_by_ids(table_name=TABLE_NAME,
                                    record_ids=ids_q1)

sims = evaluate_similarity(question,ids_data,model)
for n,i in enumerate(sims):
    print(i)
    if n > 9:
        print('')
        break

{'id': '34.230', 'content': '4.13\nWie werden in der BEG WG und BEG NWG der Anschluss an ein Gebäude- oder Wärmenetz bzw. die Errichtung von Gebäudenetzen gefördert?\n(#)\nDer Anschluss an ein Gebäude- oder Wärmenetz bzw. die Errichtung eines Gebäudenetzes werden in der BEG WG und der BEG NWG im Rahmen der Sanierung zum Effizienzhaus mitgefördert.\nBei einem Anschluss an ein Gebäudenetz wird eine EE-Klasse erreicht, wenn für die Deckung des Anteils erneuerbarer Energien (65 %) ein entsprechender Wärmeerzeuger nach den Technischen Mindestanforderungen Abschnitt 3 verwendet wird.\nDas Erreichen der EE-Klasse ist zudem durch einen Anschluss an ein Wärmenetz möglich. Hierbei gibt es keine Anforderungen an den Wärmeerzeuger des Wärmenetzes.', 'similarity_score': 0.6818543672561646}
{'id': '34.176', 'content': '(#)\nModernisierungsmaßnahmen im Bereich der Innenbeleuchtung sind als Einzelmaßnahme in Nichtwohngebäuden förderfähig. Das in der Förderrichtlinie vorgegebene Mindestinvestitionsvolu

In [9]:
tab_description = """Die Tabelle listet verschiedene Modelle von Heizkesseln des Herstellers Atmos auf. 

1. **Überblick**: Die Tabelle enthält Modelle von Atmos, die hauptsächlich in die Serien DC, GSX und P unterteilt sind. Diese Modelle sind mit Partikelabscheidern ausgestattet und teilweise für Kombianlagen geeignet.

2. **Modellvariationen**: Es gibt signifikante Unterschiede in den Modellen, wie z.B. DC 18 GSE-PA, DC 25 GSP-PA, GSX 50-PA, P 14 compact-PA, P 25-PA, etc. Die Modelle unterscheiden sich durch ihre spezifischen Typenbezeichnungen und zusätzliche Merkmale wie "mit Partikelabscheider" oder "autom. beschickter Anlagenteil".

3. **Herstellerdiversität**: Alle Modelle in der Tabelle stammen vom Hersteller Atmos.

4. **Wertbereiche**:
   - **ETA (%)**: 81,0% bis 83,0%
   - **Staub bei Nennlast (mg/m³)**: 0,8 bis 1,4
   - **Nennwärmeleistung (kW)**: 14,0 bis 49,0
   - **Kesselwirkungsgrad (%)**: 90,7% bis 92,8%
   - **CO bei Nennlast (mg/m³)**: 23,0 bis 167,0
   - **CO bei Teillast (mg/m³)**: 71,0 bis 179,0

5. **Zertifizierungen und einzigartige Attribute**: Die meisten Modelle sind mit Partikelabscheidern ausgestattet, was auf eine besondere Umweltfreundlichkeit hinweist. Es gibt keine spezifischen Zertifizierungen wie BAFA in der Tabelle erwähnt.

Die Tabelle bietet eine detaillierte Übersicht über die technischen Spezifikationen und Unterschiede der verschiedenen Heizkesselmodelle von Atmos."""

desc1 = """Die Tabelle listet verschiedene Modelle von Heizkesseln des Herstellers Atmos auf. 

1. **Überblick**: Die Tabelle enthält Modelle von Atmos, die hauptsächlich in die Serien DC, GSX und P unterteilt sind. Diese Modelle sind mit Partikelabscheidern ausgestattet und teilweise für Kombianlagen geeignet.

2. **Modellvariationen**: Es gibt signifikante Unterschiede in den Modellen, wie z.B. DC 18 GSE-PA, DC 25 GSP-PA, GSX 50-PA, P 14 compact-PA, P 25-PA, etc. Die Modelle unterscheiden sich durch ihre spezifischen Typenbezeichnungen und zusätzliche Merkmale wie "mit Partikelabscheider" oder "autom. beschickter Anlagenteil".

3. **Herstellerdiversität**: Alle Modelle in der Tabelle stammen vom Hersteller Atmos."""

desc2 = """4. **Wertbereiche**:
   - **ETA (%)**: 81,0% bis 83,0%
   - **Staub bei Nennlast (mg/m³)**: 0,8 bis 1,4
   - **Nennwärmeleistung (kW)**: 14,0 bis 49,0
   - **Kesselwirkungsgrad (%)**: 90,7% bis 92,8%
   - **CO bei Nennlast (mg/m³)**: 23,0 bis 167,0
   - **CO bei Teillast (mg/m³)**: 71,0 bis 179,0

5. **Zertifizierungen und einzigartige Attribute**: Die meisten Modelle sind mit Partikelabscheidern ausgestattet, was auf eine besondere Umweltfreundlichkeit hinweist. Es gibt keine spezifischen Zertifizierungen wie BAFA in der Tabelle erwähnt."""

desc3 = """Die Tabelle bietet eine detaillierte Übersicht über die technischen Spezifikationen und Unterschiede der verschiedenen Heizkesselmodelle von Atmos."""

sentence_pairs = [
    ("Welche technischen Merkmale unterscheiden die verschiedenen Heizkesselmodelle von Atmos in der Tabelle, und wie variieren diese in Bezug auf Wirkungsgrad, CO-Emissionen und Nennwärmeleistung?", desc1),
    ("Welche technischen Merkmale unterscheiden die verschiedenen Heizkesselmodelle von Atmos in der Tabelle, und wie variieren diese in Bezug auf Wirkungsgrad, CO-Emissionen und Nennwärmeleistung?", desc2),
    ("Welche technischen Merkmale unterscheiden die verschiedenen Heizkesselmodelle von Atmos in der Tabelle, und wie variieren diese in Bezug auf Wirkungsgrad, CO-Emissionen und Nennwärmeleistung?", desc3),
("Welche Bedeutung haben die Partikelabscheider bei den in der Tabelle aufgeführten Atmos-Modellen, und inwiefern tragen sie zur Umweltfreundlichkeit bei?",desc1),
("Welche Bedeutung haben die Partikelabscheider bei den in der Tabelle aufgeführten Atmos-Modellen, und inwiefern tragen sie zur Umweltfreundlichkeit bei?",desc2),
("Welche Bedeutung haben die Partikelabscheider bei den in der Tabelle aufgeführten Atmos-Modellen, und inwiefern tragen sie zur Umweltfreundlichkeit bei?",desc3),
("In welchen Serien sind die Atmos-Heizkessel in der Tabelle unterteilt, und welche spezifischen Modelle gehören zu diesen Serien?",desc1),  # Expected to be dissimilar
("In welchen Serien sind die Atmos-Heizkessel in der Tabelle unterteilt, und welche spezifischen Modelle gehören zu diesen Serien?",desc2),  # Expected to be dissimilar
("In welchen Serien sind die Atmos-Heizkessel in der Tabelle unterteilt, und welche spezifischen Modelle gehören zu diesen Serien?",desc3),  # Expected to be dissimilar
("Wie schneiden die Atmos-Heizkesselmodelle in Bezug auf Staubemissionen bei Nennlast ab, und welche Modelle haben die niedrigsten Emissionswerte?",desc1),
("Wie schneiden die Atmos-Heizkesselmodelle in Bezug auf Staubemissionen bei Nennlast ab, und welche Modelle haben die niedrigsten Emissionswerte?",desc2),
("Wie schneiden die Atmos-Heizkesselmodelle in Bezug auf Staubemissionen bei Nennlast ab, und welche Modelle haben die niedrigsten Emissionswerte?",desc3),
("Gibt es signifikante Unterschiede in den Zertifizierungen der Heizkesselmodelle in der Tabelle, und welche Rolle spielen diese für die Umweltfreundlichkeit der Geräte?",desc1),
("Gibt es signifikante Unterschiede in den Zertifizierungen der Heizkesselmodelle in der Tabelle, und welche Rolle spielen diese für die Umweltfreundlichkeit der Geräte?",desc2),
("Gibt es signifikante Unterschiede in den Zertifizierungen der Heizkesselmodelle in der Tabelle, und welche Rolle spielen diese für die Umweltfreundlichkeit der Geräte?",desc3)
]

model = SentenceTransformer("aari1995/German_Semantic_V3", trust_remote_code=True)

# Call the evaluation function and get the results
results = evaluate_embeddings(model, sentence_pairs)

# Print the results for manual inspection
for result in results:
    print(f"Sentence 1: {result['sentence1']}")
    print(f"Sentence 2: {result['sentence2']}")
    print(f"Similarity Score: {result['similarity']:.4f}\n")

Sentence 1: Welche technischen Merkmale unterscheiden die verschiedenen Heizkesselmodelle von Atmos in der Tabelle, und wie variieren diese in Bezug auf Wirkungsgrad, CO-Emissionen und Nennwärmeleistung?
Sentence 2: Die Tabelle listet verschiedene Modelle von Heizkesseln des Herstellers Atmos auf. 

1. **Überblick**: Die Tabelle enthält Modelle von Atmos, die hauptsächlich in die Serien DC, GSX und P unterteilt sind. Diese Modelle sind mit Partikelabscheidern ausgestattet und teilweise für Kombianlagen geeignet.

2. **Modellvariationen**: Es gibt signifikante Unterschiede in den Modellen, wie z.B. DC 18 GSE-PA, DC 25 GSP-PA, GSX 50-PA, P 14 compact-PA, P 25-PA, etc. Die Modelle unterscheiden sich durch ihre spezifischen Typenbezeichnungen und zusätzliche Merkmale wie "mit Partikelabscheider" oder "autom. beschickter Anlagenteil".

3. **Herstellerdiversität**: Alle Modelle in der Tabelle stammen vom Hersteller Atmos.
Similarity Score: 0.6699

Sentence 1: Welche technischen Merkmale unte